In [ ]:
### SETUP

import ast
import pandas as pd
import plotly.express as px
import json

# Pool Meta-data
def load_pools(json_filepath):
    """Load pool data from json (The Graph)"""
    with open(json_filepath, "r", encoding="utf-8") as f:
        return json.load(f)["data"]["pools"]

pools = load_pools("../unichain_v4_pools.json")
unique_tokens = []
for pool in pools:
    token0 = pool["token0"]
    token1 = pool["token1"]
    unique_tokens.append({"symbol": token0["symbol"], "decimals": int(token0["decimals"])})
    unique_tokens.append({"symbol": token1["symbol"], "decimals": int(token1["decimals"])})
df_tokens = pd.DataFrame(unique_tokens).drop_duplicates().reset_index(drop=True)
df_tokens.set_index("symbol", inplace=True)

In [ ]:
### UNICHAIN

df_unichain = pd.read_csv("/home/tobias/personal-dex-trading/out/data/latest_unichain_uniswap_v4.csv")
df_unichain["timestamp"] = pd.to_datetime(df_unichain["timestamp"], unit="s")  # seconds -> datetime
df_unichain["timestamp"] += pd.Timedelta(hours=2)
df_unichain["bid"] = pd.to_numeric(df_unichain["bid"], errors="coerce")
df_unichain["ask"] = pd.to_numeric(df_unichain["ask"], errors="coerce")

# transform token1
df_unichain["bid_transformed"] = df_unichain.apply(
    lambda row: row["bid"] / (10 ** df_tokens.loc[row["token1_symbol"], "decimals"]), axis=1
)
df_unichain["ask_transformed"] = df_unichain.apply(
    lambda row: row["ask"] / (10 ** df_tokens.loc[row["token1_symbol"], "decimals"]), axis=1
)

# transform token0
df_unichain["amount_in_transformed"] = df_unichain.apply(
    lambda row: row["amount_in"] / (10 ** df_tokens.loc[row["token0_symbol"], "decimals"]), axis=1
)

# normalize bid / ask
df_unichain["bid_trans_norm"] = df_unichain["bid_transformed"] / df_unichain["amount_in_transformed"]
df_unichain["ask_trans_norm"] = df_unichain["ask_transformed"] / df_unichain["amount_in_transformed"]

# tokenpair
df_unichain["token_pair"] = df_unichain["token0_symbol"].str.lower()+ df_unichain["token1_symbol"].str.lower()
# only take one amount_in
df_unichain = df_unichain.groupby(["pool_id", "block_number"]).first().reset_index()
print(f"len(df_unichain): {len(df_unichain)}")
df_unichain.head(2)

In [ ]:
### BINANCE

BINANCE_FEE = 0.001 # 0.1% for Regular User https://www.binance.com/en/fee/trading
BINANCE_FEE_VIP9 = 0.00011

def _top_price(side_str: str) -> float | None:
    """Return the first price in a stringified list of bids/asks."""
    try:
        side = ast.literal_eval(side_str)
        return float(side[0][0]) if side else None
    except Exception:
        return None

def load_binance(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)           # columns: time, bids, asks
    df["best_bid"] = df["bids"].apply(_top_price)
    df["best_ask"] = df["asks"].apply(_top_price)

    return df[["time", "token_pair", "best_bid", "best_ask"]].sort_values("time")

def apply_fee(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds columns 'best_bid_with_fee' and 'best_ask_with_fee' to account for the Binance trading fee.
    """
    df["best_bid_with_fee"] = df["best_bid"] * (1 - BINANCE_FEE)  # Subtract fee for bids
    df["best_ask_with_fee"] = df["best_ask"] * (1 + BINANCE_FEE)  # Add fee for asks
    df["best_bid_with_fee_VIP9"] = df["best_bid"] * (1 - BINANCE_FEE_VIP9)  # Subtract fee for bids
    df["best_ask_with_fee_VIP9"] = df["best_ask"] * (1 + BINANCE_FEE_VIP9)  # Add fee for asks

    return df

def invert_and_transform(df):
    """
    Invert best_bid and best_ask, and reverse token_pair, adjust to uniswap v4
    """
    inversion_mapping = {
        'wbtceth':'ethwbtc',
        'uniusdc': 'usdcuni',
        'unieth': 'ethuni',
    }

    mask = df["token_pair"].isin(inversion_mapping.keys())
    df.loc[mask, "best_bid"], df.loc[mask, "best_ask"] = (
        1 / df.loc[mask, "best_ask"],  # Invert ask to become the new bid
        1 / df.loc[mask, "best_bid"],  # Invert bid to become the new ask
    )
    df.loc[mask, "token_pair"] = df.loc[mask, "token_pair"].map(inversion_mapping)
    return df


binance_df = load_binance("/home/tobias/personal-dex-trading/out/data/latest_binance_ws_orderbook.csv")
binance_df["time"] = pd.to_datetime(binance_df["time"])
binance_df = invert_and_transform(binance_df)
binance_df = apply_fee(binance_df)

print(f"len(df_unichain): {len(binance_df)}")
binance_df.head(2)

In [ ]:
### MERGE

def create_merged_dataframe(df_unichain, binance_df):
    df_unichain["timestamp"] = pd.to_datetime(df_unichain["timestamp"])
    binance_df["time"] = pd.to_datetime(binance_df["time"])

    df_merged = pd.merge(
        df_unichain,
        binance_df,
        how="inner",
        left_on=["token_pair", "timestamp"],
        right_on=["token_pair", "time"]
    )

    df_merged = df_merged[[
        "time", "token_pair",
        "best_bid_with_fee",
        "best_ask_with_fee",
        "bid_trans_norm",
        "ask_trans_norm"
    ]]

    # Rename columns where necessary for clarity
    df_merged = df_merged.rename(columns={
        "bid_trans_norm": "unichain_bid_trans_norm",
        "ask_trans_norm": "unichain_ask_trans_norm",
        "best_bid_with_fee": "binance_best_bid_with_fee",
        "best_ask_with_fee": "binance_best_ask_with_fee",
    })

    return df_merged
merged_df = create_merged_dataframe(df_unichain, binance_df)

# add arbitrage columns
merged_df["arbitrage_buy_binance_sell_unichain"] = (
    merged_df["unichain_bid_trans_norm"] - merged_df["binance_best_ask_with_fee"]
)
merged_df["arbitrage_buy_unichain_sell_binance"] = (
    merged_df["binance_best_bid_with_fee"] - merged_df["unichain_ask_trans_norm"]
)
merged_df["arbitrage_opportunity"] = (
    (merged_df["arbitrage_buy_binance_sell_unichain"] > 0) |
    (merged_df["arbitrage_buy_unichain_sell_binance"] > 0)
)

print(f"len(df_unichain): {len(merged_df)}")
merged_df.head(2)

In [ ]:
### ARBITRAGE ANALYSIS
print(f"token_pair with arbitrage_opportunity=True:\n{merged_df[merged_df['arbitrage_opportunity']].token_pair.unique()}")

In [ ]:
merged_df.iloc[1285:1290]

In [ ]:
merged_df[(merged_df["token_pair"]=="usdcuni")&(merged_df["arbitrage_opportunity"])]["arbitrage_buy_binance_sell_unichain"].max()